# Battery Forecasting with DeepAR

In this notebook we'll prevent battery outages using Amazon Sagemaker and [DeepAR Forecasting](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html).

"The Amazon SageMaker DeepAR forecasting algorithm is a supervised learning algorithm for forecasting scalar (one-dimensional) time series using recurrent neural networks (RNN)."

Start by loading the required libraries and recovering stored data:

In [ ]:
import pandas as pd
from matplotlib import pyplot

In [ ]:
%store -r data

## Exploratory Data Analysis

Visualize the battery time series for a single device:

In [ ]:
device_loc = 1
sample_device_id = data.iloc[device_loc]["device_id"]
sample_device_id

In [ ]:
sample_data = data[data["device_id"] == sample_device_id]

In [ ]:
battery = sample_data["battery"]

In [ ]:
battery.plot()
pyplot.show()

# The DeepAR Algorithm

Autoregressive time series algorithms are a class of statistical models used for forecasting future points in a time series data set. They operate under the assumption that the value at a current time point is a function of a certain number of preceding values, plus some error term. The "auto" in autoregressive indicates that the regression is of the variable against itself, shifted by a step or more steps in time.

DeepAR is a probabilistic forecasting algorithm with a deep learning approach, developed by Amazon for the AWS platform. Unlike traditional autoregressive models that estimate a single future point, DeepAR predicts the full probability distribution of a future point in the time series. This is advantageous because it provides a quantifiable measure of uncertainty in the predictions.

DeepAR utilizes a recurrent neural network (RNN) architecture, typically with Long Short-Term Memory (LSTM) units or Gated Recurrent Units (GRUs), which are well-suited for learning patterns in sequence data. The network is trained on multiple time series data, learning shared representations that improve forecasting accuracy, especially with datasets that have complex, non-linear patterns and relationships that are difficult to model with traditional statistical methods.

https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html



## Time Series Resampling

The [DeepAR input format](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html#deepar-inputoutput) requires data to be sampled at regular time intervals and format as JSON Lines. 

Here is a sample input:


```
{"start": "2009-11-01 00:00:00", "target": [4.3, "NaN", 5.1, ...], "cat": [0, 1], "dynamic_feat": [[1.1, 1.2, 0.5, ...]]}
{"start": "2012-01-30 00:00:00", "target": [1.0, -5.0, ...], "cat": [2, 3], "dynamic_feat": [[1.1, 2.05, ...]]}
{"start": "1999-01-30 00:00:00", "target": [2.0, 1.0], "cat": [1, 4], "dynamic_feat": [[1.3, 0.4]]}

```

In the dataset, we can see that the sample timestamps are no regularly spaced, but actualy reflects the observation time. This is very common, as device activation may vary according to many factors, such as battery power, user configuration, time of day and so on.

In [ ]:
battery.tail(50).plot(style="k.")

We can observe that data is taken every 5 minutes, more or less.

In [ ]:
battery.tail(10)

Pandas offers a convenient resampling function `resample()` to create a uniform hourly dataset.
We'll also filter out zero values and take the minimal value at each hour.

In [ ]:
hourly = data[data["battery"] > 0]
hourly = (hourly.groupby("device_id")
          .battery
          .resample("h")
          .min())

In [ ]:
hourly

In [ ]:
hourly = hourly.reset_index().set_index("timestamp")

In [ ]:
hourly

Let's again visualize a sample tame series:

In [ ]:
hsample = hourly[hourly["device_id"] == sample_device_id]

In [ ]:
hsample.tail(50).plot(style="k.")

In [ ]:
hsample.plot()

## Cross Validation

Take the last hours in the dataset for testing against predictions. This lets you evaluate how your model will perform on new data.

In [ ]:
last_time = hourly.tail(1).index[0]
last_time

In [ ]:
cut_time = last_time - pd.Timedelta('8 hour')
cut_time

In [ ]:
train_set = hourly.loc[hourly.index <= cut_time]
train_set.tail()

In [ ]:
test_set = hourly.loc[hourly.index > cut_time]
test_set.head()

In [ ]:
sample_train = train_set[train_set["device_id"] == sample_device_id]["battery"]
sample_test = test_set[test_set["device_id"] == sample_device_id]

In [ ]:
sample_train.tail()

In [ ]:
sample_test.head()

In [ ]:
ax = sample_train.plot()
sample_test.plot(ax=ax)

## DeepAR Data Formatting

Convert the data from pandas DataFrame to the expected JSON Lines:

In [ ]:
import json
import math

def df_to_tss(dataframe):
    df = dataframe.copy()
    df["timeindex"] = df.index
    cats = {}
    tss = {}
    for index, row in df.iterrows():
        target = row["battery"]
        if not(math.isnan(target)):
            identity = row["device_id"]
            cat = cats.get(identity)
            if not cat:
                cat = len(cats)
                start = str(row["timeindex"])
                ts = {
                    "start": start,
                    "cat": [cat],
                    "target": [],
                }
                cats[identity] = cat
                tss[cat] = ts
            ts = tss.get(cat)
            ts["target"].append(target)
    return tss

def tss_to_jsonl(tss):  
    result = ""
    for key, value in tss.items():
        jsonll = json.dumps(value)
        result += jsonll
        result += "\n"
    return result[:-1]

def df_to_jsonl(dataframe):
    return tss_to_jsonl(df_to_tss(dataframe))

In [ ]:
import time
start = time.time()
jsonl = df_to_jsonl(train_set.head(100))
elapsed = time.time() - start
print(elapsed)
print(jsonl)

In [ ]:
import time
start = time.time()

train_tss = df_to_tss(train_set)
train_jsonl = tss_to_jsonl(train_tss)

test_tss = df_to_tss(test_set)
test_jsonl = tss_to_jsonl(test_tss)

elapsed = time.time() - start
print(elapsed)

Save the json lines files locally:

In [ ]:
import pathlib

prefix = "mt-battery-deepar"
input_path = "./{}/input".format(prefix)

train_path = "{}/train.json".format(input_path)
test_path = "{}/test.json".format(input_path)
(train_path,test_path)

In [ ]:
import shutil

shutil.rmtree(input_path, ignore_errors=True)
pathlib.Path(input_path).mkdir(parents=True, exist_ok=True)

with open(train_path, "w") as text_file:
    print(train_jsonl, file=text_file)

with open(test_path, "w") as text_file:
    print(test_jsonl, file=text_file)

In [ ]:
! ls -liah "{input_path}/"

Upload train and test sets to S3:

In [ ]:
%store -r bucket

In [ ]:
!aws s3 sync "{input_path}/" "s3://{bucket}/{prefix}/" --delete

In [ ]:
!aws s3 ls "s3://{bucket}/{prefix}/" 

In [ ]:
dar_input = {
    "train": "s3://{}/{}/train.json".format(bucket,prefix),
    "test": "s3://{}/{}/test.json".format(bucket,prefix)
}
dar_input

## DeepAR Training

The different [ML instance types](https://aws.amazon.com/sagemaker/pricing/instance-types/) in training lets you control how efficiently models learn.

"You can train DeepAR on both GPU and CPU instances and in both single and multi-machine settings. We recommend starting with a single CPU instance (for example, ml.c4.2xlarge or ml.c4.4xlarge), and switching to GPU instances and multiple machines only when necessary."

In [ ]:
# train_instance_type='ml.c5.2xlarge' Estimated Training Time: 10m
train_instance_type='ml.c5.2xlarge'

In [ ]:
import boto3
import sagemaker
from sagemaker.amazon.amazon_estimator import get_image_uri

dar_image_name = sagemaker.image_uris.retrieve('forecasting-deepar', boto3.Session().region_name)
# dar_image_name = get_image_uri(boto3.Session().region_name, 'forecasting-deepar')
dar_image_name

In [ ]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

dar_estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_uri=dar_image_name,
    role=role,
    instance_count=1,
    instance_type=train_instance_type,
    base_job_name=prefix,
    output_path="s3://{}/{}/output/".format(bucket,prefix)
)

In [ ]:
Tune the following hyperparameters to the application:

In [ ]:
freq = 'H'
prediction_length = 4
context_length = 12

Here are some suggestions for the other parameters. Feel free to try other seetings!

In [ ]:

dar_hyperparameters = {
    "time_freq": freq,
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
    "num_cells": "40",
    "num_layers": "3",
    "likelihood": "gaussian",
    "epochs": "10",
    "mini_batch_size": "32",
    "learning_rate": "0.001",
    "dropout_rate": "0.05",
    "early_stopping_patience": "10",
    "cardinality": "auto",
    "num_dynamic_feat":"ignore"
}
dar_estimator.set_hyperparameters(**dar_hyperparameters)

In [ ]:
dar_estimator.fit(inputs=dar_input)

## How is a neural network trained?

### Backpropagation

![NeuralNetTraining](./img/nn_training.gif)

### LSTM



In [ ]:
dar_job_name = dar_estimator.latest_training_job.name
dar_job_name

## DeepAR Inference

In [ ]:
infer_instance_type="ml.c5.2xlarge"

In [ ]:
dar_endpoint_name = sagemaker_session.endpoint_from_job(
    job_name=dar_job_name,
    initial_instance_count=1,
    instance_type=infer_instance_type,
    image_uri=dar_image_name,
    role=role
)
dar_endpoint_name

In [ ]:
instances = []

In [ ]:
instances.append(train_tss[1])

In [ ]:
inference = {
    "instances": instances,
    "configuration": {
         "output_types": ["mean", "quantiles"],
         "quantiles": ["0.1", "0.5","0.9"]
    }
}

In [ ]:
import json
inference_json = json.dumps(inference, indent=2)
print(inference_json)

In [ ]:
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import JSONSerializer



predictor = sagemaker.predictor.Predictor(
    dar_endpoint_name, 
    sagemaker_session=sagemaker_session, 
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer())
predictor

In [ ]:
prediction = predictor.predict(inference)

In [ ]:
prediction

In [ ]:
predictions = prediction["predictions"]
predictions

## DeepAR Evaluation

In [ ]:
pred0 = predictions[0]
pred0

In [ ]:
mean = pred0["mean"]
quantiles = pred0["quantiles"]
q95 = quantiles["0.5"]

In [ ]:
actual = sample_test["battery"][0:4].values
actual

In [ ]:
import matplotlib.pyplot as plt

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.grid(which='major', axis='both')

ax1.set_ylabel('Actual Battery', color='C0')
ax2.set_ylabel('Predicted Battery', color='C1')


ax1.plot(actual, color='C0')
ax2.plot(q95, color='C1')


# Hyperparameter Tuning

Hyperparameter tuning on Amazon SageMaker is a method that automates the optimization of hyperparameters to improve the performance of a machine learning model. Hyperparameters are the configuration settings used to structure the learning process, and they can significantly impact the model's accuracy. Unlike model parameters, which the model learns during training, hyperparameters are set prior to training and remain constant throughout the process.

SageMaker's hyperparameter tuning works by executing multiple training jobs in parallel, each with different sets of hyperparameters, based on the defined search space. It uses algorithms like Bayesian optimization to intelligently navigate through the search space, focusing on combinations likely to produce the best results. The service evaluates the performance of each set of hyperparameters by using a predefined metric, such as accuracy or loss, identifying the best-performing model. This automated process simplifies the otherwise manual and time-consuming task of hyperparameter optimization, making model development faster and more efficient.

Check the reference for all tunable parameters and metrics: https://docs.aws.amazon.com/sagemaker/latest/dg/deepar-tuning.html


In [ ]:
import sagemaker
from sagemaker.tuner import *

# Specify the hyperparameters to tune and their ranges
hyperparameter_ranges = {
    'epochs': IntegerParameter(4, 32),
    'context_length': IntegerParameter(4, 16),
    'num_cells': IntegerParameter(20, 120),
    'num_layers': IntegerParameter(1, 6)
}

# Define the metric to use for evaluation
objective_metric_name = 'train:final_loss'

# Configure HyperparameterTuner
tuner = HyperparameterTuner(
    dar_estimator,
    objective_metric_name,
    hyperparameter_ranges,
    max_jobs=6,
    max_parallel_jobs=3,
    strategy='Bayesian', # You can also choose 'Random' or 'Grid Search'
    objective_type='Minimize'
)

# Start the tuning job
tuner.fit(dar_input)


Once the hyperparameter tuning job finishes, let's check the resulting parameters.

In [ ]:
import sagemaker

tuner_analytics = sagemaker.HyperparameterTuningJobAnalytics(tuner.latest_tuning_job.job_name)
best_training_job_name = sagemaker.Session().sagemaker_client.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name
)['BestTrainingJob']['TrainingJobName']
hpt = hpt[hpt['TrainingJobName'] == best_training_job_name]

hpt

You can now re-train your model with the optimized hyperparameters.

# Motor Anomalies

Great job on predicting the future! 

Now let's work on detecting [motor anomalies](mt-motor-anomaly.ipynb).